<a href="https://colab.research.google.com/github/MadmanMarble/MadmanMarble/blob/main/U-net_dataGenerator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install rasterio scikit-image tensorflow keras gdown
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.6/20.6 MB 38.4 MB/s eta 0:00:00


In [3]:
import os
import glob
import gdown
import zipfile
import cv2
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import shutil
import rasterio
import rasterio.plot
from tensorflow import keras
from tensorflow.keras import layers
from keras.utils import to_categorical
from keras.models import load_model
from keras.utils import Sequence, to_categorical
from sklearn.model_selection import train_test_split
from skimage.transform import resize
from skimage.util import random_noise
from scipy import ndimage
from scipy.ndimage import label as nd_label
from scipy.ndimage import generic_filter
from scipy.stats import mode

# Additional code can be added here if needed

In [ ]:
# image_paths_2018 = glob.glob(f"/content/drive/MyDrive/DeepLearning_Project/trainingdata/planet128x128data/utm5/images_2018/*.tif")
# image_paths_2019 = glob.glob(f"/content/drive/MyDrive/DeepLearning_Project/trainingdata/planet128x128data/utm5/images_2019/*.tif")
# image_paths_2020 = glob.glob(f"/content/drive/MyDrive/DeepLearning_Project/trainingdata/planet128x128data/utm5/images_2020/*.tif")
# dem_path = glob.glob(f"/content/drive/MyDrive/DeepLearning_Project/trainingdata/planet128x128data/utm5/dem/*.tif")
# nwi_ccap_path = glob.glob(f"/content/drive/MyDrive/DeepLearning_Project/trainingdata/planet128x128data/utm5/nwi_ccap_filled/*.tif")
# label_paths = glob.glob(f"/content/drive/MyDrive/DeepLearning_Project/trainingdata/planet128x128data/utm5/ccap_filled/*.tif")

train test split

In [4]:
planet_images = glob.glob(f"/content/drive/MyDrive/DeepLearning_Project/unet_planet_data/planet_images/*.tif")

In [6]:
planet_images

['/content/drive/MyDrive/DeepLearning_Project/unet_planet_data/planet_images/1591_planet_stack_utm4.tif',
 '/content/drive/MyDrive/DeepLearning_Project/unet_planet_data/planet_images/1706_planet_stack_utm4.tif',
 '/content/drive/MyDrive/DeepLearning_Project/unet_planet_data/planet_images/1310_planet_stack_utm4.tif',
 '/content/drive/MyDrive/DeepLearning_Project/unet_planet_data/planet_images/1051_planet_stack_utm4.tif',
 '/content/drive/MyDrive/DeepLearning_Project/unet_planet_data/planet_images/1818_planet_stack_utm4.tif',
 '/content/drive/MyDrive/DeepLearning_Project/unet_planet_data/planet_images/1162_planet_stack_utm4.tif',
 '/content/drive/MyDrive/DeepLearning_Project/unet_planet_data/planet_images/1236_planet_stack_utm4.tif',
 '/content/drive/MyDrive/DeepLearning_Project/unet_planet_data/planet_images/1796_planet_stack_utm4.tif',
 '/content/drive/MyDrive/DeepLearning_Project/unet_planet_data/planet_images/1256_planet_stack_utm4.tif',
 '/content/drive/MyDrive/DeepLearning_Project/

In [5]:
planet_labels = glob.glob(f"/content/drive/MyDrive/DeepLearning_Project/unet_planet_data/planet_labels/*.tif")

In [7]:
planet_labels

['/content/drive/MyDrive/DeepLearning_Project/unet_planet_data/planet_labels/685_nwi_ccap_filled_utm4.tif',
 '/content/drive/MyDrive/DeepLearning_Project/unet_planet_data/planet_labels/525_nwi_ccap_filled_utm4.tif',
 '/content/drive/MyDrive/DeepLearning_Project/unet_planet_data/planet_labels/1145_nwi_ccap_filled_utm4.tif',
 '/content/drive/MyDrive/DeepLearning_Project/unet_planet_data/planet_labels/1248_nwi_ccap_filled_utm4.tif',
 '/content/drive/MyDrive/DeepLearning_Project/unet_planet_data/planet_labels/21_nwi_ccap_filled_utm4.tif',
 '/content/drive/MyDrive/DeepLearning_Project/unet_planet_data/planet_labels/1455_nwi_ccap_filled_utm4.tif',
 '/content/drive/MyDrive/DeepLearning_Project/unet_planet_data/planet_labels/2247_nwi_ccap_filled_utm4.tif',
 '/content/drive/MyDrive/DeepLearning_Project/unet_planet_data/planet_labels/1973_nwi_ccap_filled_utm4.tif',
 '/content/drive/MyDrive/DeepLearning_Project/unet_planet_data/planet_labels/2519_nwi_ccap_filled_utm4.tif',
 '/content/drive/MyDriv

In [8]:
from sklearn.model_selection import train_test_split
import numpy as np
import os

In [9]:

# Ensure the lists are sorted so that images and labels correspond to each other
planet_labels = sorted(glob.glob(f"/content/drive/MyDrive/DeepLearning_Project/unet_planet_data/planet_labels/*.tif"))
planet_images = sorted(glob.glob(f"/content/drive/MyDrive/DeepLearning_Project/unet_planet_data/planet_images/*.tif"))

# Split the file paths into training and testing sets (80% train, 20% test)
train_images_paths, test_images_paths, train_labels_paths, test_labels_paths = train_test_split(
    planet_images, planet_labels, test_size=0.2, random_state=42)

# Now, you can load images and labels from these paths as needed for training and testing


In [ ]:
# # Load the images and masks arrays
# train_images = np.load('/content/drive/MyDrive/DeepLearning_Project/unet_planet_data/train_images.npy')
# train_masks = np.load('/content/drive/MyDrive/DeepLearning_Project/unet_planet_data/train_masks.npy')

# # Split the arrays into training and testing sets (80% train, 20% test)
# X_train, X_test, y_train, y_test = train_test_split(
#     train_images, train_masks, test_size=0.2, random_state=42)

# # X_train, y_train are now your training set
# # X_test, y_test are your testing set


end

In [10]:
class DataGenerator(Sequence):
    def __init__(self, image_files, label_files, img_height, img_width, batch_size, num_classes):
        self.image_files = image_files
        self.label_files = label_files
        self.img_height = img_height
        self.img_width = img_width
        self.batch_size = batch_size
        self.num_classes = num_classes
        # self.noise = noise

    def __len__(self):
        return int(np.ceil(len(self.image_files) / self.batch_size))

    def __getitem__(self, index):
        image_batch_files = self.image_files[index * self.batch_size : (index + 1) * self.batch_size]
        label_batch_files = self.label_files[index * self.batch_size : (index + 1) * self.batch_size]

        batch_images, batch_labels = self.load_images_and_labels(image_batch_files,label_batch_files)
        return batch_images, batch_labels

    def load_and_reshape_image(self, image_path):
        with rasterio.open(image_path) as src:
            image = src.read()
            image = image.transpose((1, 2, 0))
            if image.shape[0] != self.img_height or image.shape[1] != self.img_width:
                image = cv2.resize(image, (self.img_width, self.img_height), interpolation=cv2.INTER_NEAREST)
            if len(image.shape) == 3 and image.shape[2] == 1:
                image = np.squeeze(image, axis=2)

            return image

    def load_images_and_labels(self, image_files, label_files):
        images = []
        labels = []

        for image_file in image_files:
            image = self.load_and_reshape_image(image_file)
            # image[image <= -3e+38] = np.nan

            # # # Replace NaN values with the mean of the non-NaN pixels
            # if np.any(np.isnan(image)):
            #     nan_mask = np.isnan(image)
            #     image[nan_mask] = np.nanmean(image)

            # # # Replace Inf values with the mean of the non-Inf pixels
            # if np.any(np.isinf(image)):
            #     inf_mask = np.isinf(image)
            #     image[inf_mask] = np.nanmean(image)

            # # Convert to float
            # image = image.astype(np.float32)

            # # Z-score normalization
            # mean = np.mean(image, axis=(0, 1), keepdims=True)
            # std = np.std(image, axis=(0, 1), keepdims=True)
            # # mean[mean < 0]
            # std[std < 0] = 0

            # # Normalize with epsilon to prevent divide by zero
            # epsilon = 1e-7

            # image = (image - mean) / (std + epsilon)

            images.append(image)

        for label_file in label_files:
            # label_file = image_file.replace("planet", "ccap_filled").replace('images_2018','ccap_filled')
            label = self.load_and_reshape_image(label_file)
            # label[label <= -3e+38] = np.nan

            # # # Replace NaN values with the mean of the non-NaN pixels
            # if np.any(np.isnan(label)):
            #     nan_mask = np.isnan(label)
            #     label[nan_mask] = 12

            # # # Replace Inf values with the mean of the non-Inf pixels
            # if np.any(np.isinf(label)):
            #     inf_mask = np.isinf(label)
            #     label[inf_mask] = 12

            label -= 1
            label = label.astype(np.int8)
            label = to_categorical(label, num_classes=self.num_classes)
            labels.append(label)

        return np.array(images), np.array(labels)

In [ ]:
# train_image_paths = glob.glob(f"/content/drive/MyDrive/DeepLearning_Project/trainingdata/planet128x128data/utm4/images_2018/*.tif")
# train_label_paths = glob.glob(f"/content/drive/MyDrive/DeepLearning_Project/trainingdata/planet128x128data/utm4/ccap_filled/*.tif")

# # val_image_paths = glob.glob(f"{base_dir}/validation/image/*.tif")
# # val_label_paths = glob.glob(f"{base_dir}/validation/label/*.tif")


# planet_images_path = split_paths(planet_images, set(train_ids))
# planet_labels_path = split_paths(planet_labels, set(train_ids))

In [23]:
import shutil
import os

# List of files to copy
# files_to_copy = ['path/to/source/file1.txt', 'path/to/source/file2.jpg', 'path/to/source/file3.pdf']
# Destination directory
destination = '/content/train_images'

# Ensure the destination directory exists
# os.makedirs(destination, exist_ok=True)

# Loop through the list and copy each file
for file in train_images_paths:
    # Define the full destination path for the current file
    dest_file = os.path.join(destination, os.path.basename(file))
    # Copy the file
    shutil.copy(file, dest_file)
    print(f'Copied {file} to {dest_file}')


Copied /content/drive/MyDrive/DeepLearning_Project/unet_planet_data/planet_images/1766_planet_stack_utm4.tif to /content/train_images/1766_planet_stack_utm4.tif
Copied /content/drive/MyDrive/DeepLearning_Project/unet_planet_data/planet_images/437_planet_stack_utm4.tif to /content/train_images/437_planet_stack_utm4.tif
Copied /content/drive/MyDrive/DeepLearning_Project/unet_planet_data/planet_images/1646_planet_stack_utm4.tif to /content/train_images/1646_planet_stack_utm4.tif
Copied /content/drive/MyDrive/DeepLearning_Project/unet_planet_data/planet_images/1576_planet_stack_utm4.tif to /content/train_images/1576_planet_stack_utm4.tif
Copied /content/drive/MyDrive/DeepLearning_Project/unet_planet_data/planet_images/1853_planet_stack_utm4.tif to /content/train_images/1853_planet_stack_utm4.tif
Copied /content/drive/MyDrive/DeepLearning_Project/unet_planet_data/planet_images/1913_planet_stack_utm4.tif to /content/train_images/1913_planet_stack_utm4.tif
Copied /content/drive/MyDrive/DeepLe

In [24]:
# Destination directory
destination = '/content/train_labels'

# Ensure the destination directory exists
os.makedirs(destination, exist_ok=True)

# Loop through the list and copy each file
for file in train_labels_paths:
    # Define the full destination path for the current file
    dest_file = os.path.join(destination, os.path.basename(file))
    # Copy the file
    shutil.copy(file, dest_file)
    print(f'Copied {file} to {dest_file}')


Copied /content/drive/MyDrive/DeepLearning_Project/unet_planet_data/planet_labels/1766_nwi_ccap_filled_utm4.tif to /content/train_labels/1766_nwi_ccap_filled_utm4.tif
Copied /content/drive/MyDrive/DeepLearning_Project/unet_planet_data/planet_labels/437_nwi_ccap_filled_utm4.tif to /content/train_labels/437_nwi_ccap_filled_utm4.tif
Copied /content/drive/MyDrive/DeepLearning_Project/unet_planet_data/planet_labels/1646_nwi_ccap_filled_utm4.tif to /content/train_labels/1646_nwi_ccap_filled_utm4.tif
Copied /content/drive/MyDrive/DeepLearning_Project/unet_planet_data/planet_labels/1576_nwi_ccap_filled_utm4.tif to /content/train_labels/1576_nwi_ccap_filled_utm4.tif
Copied /content/drive/MyDrive/DeepLearning_Project/unet_planet_data/planet_labels/1853_nwi_ccap_filled_utm4.tif to /content/train_labels/1853_nwi_ccap_filled_utm4.tif
Copied /content/drive/MyDrive/DeepLearning_Project/unet_planet_data/planet_labels/1913_nwi_ccap_filled_utm4.tif to /content/train_labels/1913_nwi_ccap_filled_utm4.tif


In [25]:
# Destination directory
destination = '/content/test_images'

# Ensure the destination directory exists
os.makedirs(destination, exist_ok=True)

# Loop through the list and copy each file
for file in test_images_paths:
    # Define the full destination path for the current file
    dest_file = os.path.join(destination, os.path.basename(file))
    # Copy the file
    shutil.copy(file, dest_file)
    print(f'Copied {file} to {dest_file}')

Copied /content/drive/MyDrive/DeepLearning_Project/unet_planet_data/planet_images/1903_planet_stack_utm4.tif to /content/test_images/1903_planet_stack_utm4.tif
Copied /content/drive/MyDrive/DeepLearning_Project/unet_planet_data/planet_images/548_planet_stack_utm4.tif to /content/test_images/548_planet_stack_utm4.tif
Copied /content/drive/MyDrive/DeepLearning_Project/unet_planet_data/planet_images/2218_planet_stack_utm4.tif to /content/test_images/2218_planet_stack_utm4.tif
Copied /content/drive/MyDrive/DeepLearning_Project/unet_planet_data/planet_images/794_planet_stack_utm4.tif to /content/test_images/794_planet_stack_utm4.tif
Copied /content/drive/MyDrive/DeepLearning_Project/unet_planet_data/planet_images/1915_planet_stack_utm4.tif to /content/test_images/1915_planet_stack_utm4.tif
Copied /content/drive/MyDrive/DeepLearning_Project/unet_planet_data/planet_images/1199_planet_stack_utm4.tif to /content/test_images/1199_planet_stack_utm4.tif
Copied /content/drive/MyDrive/DeepLearning_P

In [26]:
# Destination directory
destination = '/content/test_labels'

# Ensure the destination directory exists
os.makedirs(destination, exist_ok=True)

# Loop through the list and copy each file
for file in test_labels_paths:
    # Define the full destination path for the current file
    dest_file = os.path.join(destination, os.path.basename(file))
    # Copy the file
    shutil.copy(file, dest_file)
    print(f'Copied {file} to {dest_file}')

Copied /content/drive/MyDrive/DeepLearning_Project/unet_planet_data/planet_labels/1903_nwi_ccap_filled_utm4.tif to /content/test_labels/1903_nwi_ccap_filled_utm4.tif
Copied /content/drive/MyDrive/DeepLearning_Project/unet_planet_data/planet_labels/548_nwi_ccap_filled_utm4.tif to /content/test_labels/548_nwi_ccap_filled_utm4.tif
Copied /content/drive/MyDrive/DeepLearning_Project/unet_planet_data/planet_labels/2218_nwi_ccap_filled_utm4.tif to /content/test_labels/2218_nwi_ccap_filled_utm4.tif
Copied /content/drive/MyDrive/DeepLearning_Project/unet_planet_data/planet_labels/794_nwi_ccap_filled_utm4.tif to /content/test_labels/794_nwi_ccap_filled_utm4.tif
Copied /content/drive/MyDrive/DeepLearning_Project/unet_planet_data/planet_labels/1915_nwi_ccap_filled_utm4.tif to /content/test_labels/1915_nwi_ccap_filled_utm4.tif
Copied /content/drive/MyDrive/DeepLearning_Project/unet_planet_data/planet_labels/1199_nwi_ccap_filled_utm4.tif to /content/test_labels/1199_nwi_ccap_filled_utm4.tif
Copied /

In [27]:
test_images = sorted(glob.glob(f"/content/test_images/*.tif"))
test_labels = sorted(glob.glob(f"/content/test_labels/*.tif"))


train_images = sorted(glob.glob(f"/content/train_images/*.tif"))
train_labels = sorted(glob.glob(f"/content/train_labels/*.tif"))



In [28]:
# data_gen_test = DataGenerator(sum_df['Images_path'][0:5], sum_df['Labels_path'][0:5], 512, 512, 2, 22)
# Create the data generator

# train_images_paths, test_images_paths, train_labels_paths, test_labels_paths
training_data_generator = DataGenerator(train_images, train_labels, 128, 128, 13, 12)
test_data_generator = DataGenerator(test_images, test_labels, 128, 128, 13, 12)

# validation_data_generator = DataGenerator(sum_df['Images_path'][sum_df['random_split']==1], sum_df['Labels_path'][sum_df['random_split']==1], 512, 512, 4, 25)


In [12]:
batch_images, batch_labels = training_data_generator.__getitem__(0)

In [13]:
batch_images

array([[[[-3.4000000e+38, -3.4000000e+38, -3.4000000e+38, ...,
          -3.4000000e+38, -3.4000000e+38,  7.0000000e+00],
         [-3.4000000e+38, -3.4000000e+38, -3.4000000e+38, ...,
          -3.4000000e+38, -3.4000000e+38,  7.0000000e+00],
         [-3.4000000e+38, -3.4000000e+38, -3.4000000e+38, ...,
          -3.4000000e+38, -3.4000000e+38,  7.0000000e+00],
         ...,
         [-3.4000000e+38, -3.4000000e+38, -3.4000000e+38, ...,
          -3.4000000e+38, -3.4000000e+38,  7.0000000e+00],
         [-3.4000000e+38, -3.4000000e+38, -3.4000000e+38, ...,
          -3.4000000e+38, -3.4000000e+38,  7.0000000e+00],
         [-3.4000000e+38, -3.4000000e+38, -3.4000000e+38, ...,
          -3.4000000e+38, -3.4000000e+38,  7.0000000e+00]],

        [[-3.4000000e+38, -3.4000000e+38, -3.4000000e+38, ...,
          -3.4000000e+38, -3.4000000e+38,  6.9698672e+00],
         [-3.4000000e+38, -3.4000000e+38, -3.4000000e+38, ...,
          -3.4000000e+38, -3.4000000e+38,  7.0000000e+00],
        

In [ ]:
# def create_model(img_size,num_bands, num_classes):
#     inputs = keras.Input(shape=(img_size[0], img_size[1], num_bands))  # Change the number of channels to 4

#     # Entry block
#     x = layers.Conv2D(32, 3, strides=2, padding="same")(inputs)
#     x = layers.BatchNormalization()(x)
#     x = layers.Activation("LeakyReLU")(x)

#     previous_block_activation = x  # Set aside residual

#     # Blocks 1, 2, 3 are identical apart from the feature depth.
#     for filters in [64, 128, 256]:
#         x = layers.Activation("LeakyReLU")(x)
#         x = layers.SeparableConv2D(filters, 3, padding="same")(x)
#         x = layers.BatchNormalization()(x)

#         x = layers.Activation("LeakyReLU")(x)
#         x = layers.SeparableConv2D(filters, 3, padding="same")(x)
#         x = layers.BatchNormalization()(x)

#         x = layers.MaxPooling2D(3, strides=2, padding="same")(x)

#         # Project residual
#         residual = layers.Conv2D(filters, 1, strides=2, padding="same")(
#             previous_block_activation
#         )
#         x = layers.add([x, residual])  # Add back residual
#         previous_block_activation = x  # Set aside next residual

#     for filters in [256, 128, 64, 32]:
#         x = layers.Activation("LeakyReLU")(x)
#         x = layers.Conv2DTranspose(filters, 3, padding="same")(x)
#         x = layers.BatchNormalization()(x)

#         x = layers.Activation("LeakyReLU")(x)
#         x = layers.Conv2DTranspose(filters, 3, padding="same")(x)
#         x = layers.BatchNormalization()(x)

#         x = layers.UpSampling2D(2)(x)

#         residual = layers.UpSampling2D(2)(previous_block_activation)
#         residual = layers.Conv2D(filters, 1, padding="same")(residual)
#         x = layers.add([x, residual])  # Add back residual
#         previous_block_activation = x  # Set aside next residual

#     # Add a per-pixel classification layer
#     outputs = layers.Conv2D(num_classes, 3, activation="softmax", padding="same")(x)

#     # Define the model
#     model = keras.Model(inputs, outputs)
#     model.compile(optimizer="adam", loss='sparse_categorical_crossentropy', metrics=['accuracy'])  # updated to sparse categorical cross-entropy loss
#     return model

In [14]:

# from tensorflow.python.keras.layers.normalization import BatchNormalization
# from tensorflow.python.keras.layers.normalization import BatchNormalization

# from tensorflow.keras.layers import BatchNormalization
# from keras.layers.normalization.batch_normalization import BatchNormalization
# from tensorflow.python.keras.layers import BatchNormalization

# U-Net model for image segmentation.
# Encoder and decoder conncted by a center block.
# Encoder downsamples the input image while capturing its features.
# Decoder upsamples the encoded image to generate a segmentation map.

def conv_block(input_tensor, num_filters):
	encoder = layers.Conv2D(num_filters, (3, 3), padding='same')(input_tensor)
	encoder = layers.BatchNormalization()(encoder)
	encoder = layers.Activation('LeakyReLU')(encoder) # from relu to LeakyReLU
	encoder = layers.Conv2D(num_filters, (3, 3), padding='same')(encoder)
	encoder = layers.BatchNormalization()(encoder)
	encoder = layers.Activation('LeakyReLU')(encoder) # from relu to LeakyReLU
	return encoder

def encoder_block(input_tensor, num_filters):
	encoder = conv_block(input_tensor, num_filters)
	encoder_pool = layers.MaxPooling2D((2, 2), strides=(2, 2))(encoder)
	return encoder_pool, encoder

def decoder_block(input_tensor, concat_tensor, num_filters):
	decoder = layers.Conv2DTranspose(num_filters, (2, 2), strides=(2, 2), padding='same')(input_tensor)
	decoder = layers.concatenate([concat_tensor, decoder], axis=-1)
	decoder = layers.BatchNormalization()(decoder)
	decoder = layers.Activation('LeakyReLU')(decoder) # from relu to LeakyReLU
	decoder = layers.Conv2D(num_filters, (3, 3), padding='same')(decoder)
	decoder = layers.BatchNormalization()(decoder)
	decoder = layers.Activation('LeakyReLU')(decoder) # from relu to LeakyReLU
	decoder = layers.Conv2D(num_filters, (3, 3), padding='same')(decoder)
	decoder = layers.BatchNormalization()(decoder)
	decoder = layers.Activation('LeakyReLU')(decoder) # from relu to LeakyReLU
	return decoder

def get_model():
	inputs = layers.Input(shape=[KERNEL_SIZE, KERNEL_SIZE, len(BANDS)]) # 256
	encoder0_pool, encoder0 = encoder_block(inputs, 32) # 128
	encoder1_pool, encoder1 = encoder_block(encoder0_pool, 64) # 64
	encoder2_pool, encoder2 = encoder_block(encoder1_pool, 128) # 32
	encoder3_pool, encoder3 = encoder_block(encoder2_pool, 256) # 16
	encoder4_pool, encoder4 = encoder_block(encoder3_pool, 512) # 8
	center = conv_block(encoder4_pool, 1024) # center
	decoder4 = decoder_block(center, encoder4, 512) # 16
	decoder3 = decoder_block(decoder4, encoder3, 256) # 32
	decoder2 = decoder_block(decoder3, encoder2, 128) # 64
	decoder1 = decoder_block(decoder2, encoder1, 64) # 128
	decoder0 = decoder_block(decoder1, encoder0, 32) # 256
	outputs = layers.Conv2D(CLASSES, (1, 1), activation='softmax')(decoder0) # kept because this is a multiclass classification

	model = models.Model(inputs=[inputs], outputs=[outputs])

	model.compile(
		optimizer=optimizers.get(OPTIMIZER),
		loss=losses.get(LOSS),
		metrics=[metrics.get(metric) for metric in METRICS])

	return model

In [15]:
KERNEL_SIZE = 128
CLASSES = 12
BANDS = range(13)
KERNEL_SHAPE = [128, 128]
OPTIMIZER = 'adam'
LOSS = 'categorical_crossentropy'
METRICS = ['categorical_accuracy']

In [16]:
# import TenserFlow classes and functions
from tensorflow.keras import layers
from tensorflow.keras import losses
from tensorflow.keras import models
from tensorflow.keras import metrics
from tensorflow.keras import optimizers
from tensorflow.keras.layers import BatchNormalization

In [17]:
# model = create_model([128,128],4,11)
model = get_model()

In [18]:
print(model.summary()) # 31,127,148

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 128, 128, 13)]       0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 128, 128, 32)         3776      ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 128, 128, 32)         128       ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     (None, 128, 128, 32)         0         ['batch_normalization[0][0

In [29]:
# Train the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy','categorical_accuracy'])
callbacks = [
    keras.callbacks.ModelCheckpoint("landcover_segmentation.h5", save_best_only=True)
]

model.fit(training_data_generator, validation_data= test_data_generator, epochs=10, callbacks=callbacks,shuffle=True)

Epoch 1/10
174/174 [==============================] - 223s 1s/step - loss: nan - accuracy: 0.2193 - categorical_accuracy: 0.2193 - val_loss: nan - val_accuracy: 0.2090 - val_categorical_accuracy: 0.2090
Epoch 2/10
174/174 [==============================] - 203s 1s/step - loss: nan - accuracy: 0.2193 - categorical_accuracy: 0.2193 - val_loss: nan - val_accuracy: 0.2090 - val_categorical_accuracy: 0.2090
Epoch 3/10
 15/174 [=>............................] - ETA: 2:29 - loss: nan - accuracy: 0.2418 - categorical_accuracy: 0.2418

KeyboardInterrupt: 